In [ ]:
import numpy as np
import pandas as pd

In [ ]:
activity = pd.read_csv("test_activity.csv")
guild = pd.read_csv("test_guild.csv")
party = pd.read_csv("test_party.csv")
payment = pd.read_csv("test_payment.csv")
payment2 = pd.read_csv("test_payment2.csv")
trade = pd.read_csv("test_trade.csv")

In [ ]:
label = activity.groupby("acc_id").count().reset_index()[["acc_id"]]
label.head()


In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

## activity

In [ ]:
activity = activity.drop("Unnamed: 0", axis=1)

print(len(activity))
activity.head()

## guild

In [ ]:
guild = guild.drop("Unnamed: 0", axis=1)

print(len(guild))
guild.head()

## party

In [ ]:
party = party.drop("Unnamed: 0", axis=1)

print(len(party))
party.head()

In [ ]:
label2 = label.copy()
party2 = party.rename(columns = {"hashed" : "acc_id"})

In [ ]:
party2 = party2.sort_values(by = ["acc_id", "party_start_week", "party_start_day", "party_start_time"])
label2 = label2.sort_values(by = "acc_id")

In [ ]:
party2 = party2[["acc_id", "party_start_week", "party_start_day", "party_start_time", "party_end_week", "party_end_day", "party_end_time"]]
party2.head(40)

In [ ]:
label2.head()

In [ ]:
party3 = pd.merge(label2, party2, how='left', on=['acc_id'])

print(len(party3))
party3 .head(40)

In [ ]:
print(party3.groupby("acc_id")["party_start_week"].count().min())
print(party3.groupby("acc_id")["party_start_week"].count().mean())
print(party3.groupby("acc_id")["party_start_week"].count().median())
print(party3.groupby("acc_id")["party_start_week"].count().max())

In [ ]:
#party 횟수의 분포
print(len(party3.groupby("acc_id").count()))
party3.groupby("acc_id").count().groupby("party_start_week").count()

## payment

In [ ]:
payment = payment.drop("Unnamed: 0", axis=1).rename(columns = {"payment_week":"week"})

print(len(payment))
payment.head()

In [ ]:
print(len(payment2))
payment2.head()

## trade

In [ ]:
trade = trade.drop("Unnamed: 0", axis=1)
trade["source"] = 1
trade["target"] = 0

print(len(trade))
trade.head()

In [ ]:
trade1 = trade[["source_acc_id", "trade_week","trade_day","trade_time","item_type", "item_amount", "source"]].rename(columns = {"source_acc_id":"acc_id"})
trade2 = trade[["target_acc_id", "trade_week","trade_day","trade_time","item_type", "item_amount", "target"]].rename(columns = {"target_acc_id":"acc_id", "target":"source"})

In [ ]:
trade1 = trade1.sort_values(by = ["acc_id", "trade_week", "trade_day", "trade_time"])
trade2 = trade2.sort_values(by = ["acc_id", "trade_week", "trade_day", "trade_time"])

In [ ]:
trade3 = pd.concat([trade1, trade2]).sort_values(by = ["acc_id", "trade_week", "trade_day", "trade_time"])

print(len(trade3))
trade3.head(10)

In [ ]:
trade4 = trade3.drop_duplicates(subset=["acc_id", "trade_week","trade_day","trade_time","item_type", "item_amount", "source"])

print(len(trade4))
trade4.head(20)

In [ ]:
trade5 = pd.merge(label2, trade4, how='left', on=['acc_id'])

print(len(trade5))
trade5.head(40)

In [ ]:
#trade 횟수의 분포
temp = trade5.groupby("acc_id")["trade_week"].count()
print(temp.min())
print(temp.mean())
print(temp.median())
print(temp.max())

In [ ]:
#trade 횟수의 분포
temp = trade5.groupby("acc_id").count()
print(len(temp))
temp.groupby("trade_week").count()

<br></br><br></br><br></br>

# activity vector 만들기

In [ ]:
trade5.head()

In [ ]:
party3.head()

In [ ]:
trade6 = trade5.rename(columns = {"trade_week" : "week", "trade_day" : "day"})
trade6["hour"] = trade6["trade_time"].map(lambda x : int(x.split(":")[0]) if type(x) == str else x)
trade6["minute"] = trade6["trade_time"].map(lambda x : int(x.split(":")[1]) if type(x) == str else x)
trade6 = trade6.drop("trade_time", axis=1)

In [ ]:
trade6 = trade6.sort_values(by = ["acc_id", "week", "day", "hour", "minute"])[["acc_id", "week", "day", "hour", "minute", "item_type", "item_amount", "source"]]
trade6.head()

In [ ]:
party4 = party3.rename(columns = {"party_start_week" : "week", "party_start_day" : "day"})
party4["hour"] = party4["party_start_time"].map(lambda x : int(x.split(":")[0]) if type(x) == str else x)
party4["minute"] = party4["party_start_time"].map(lambda x : int(x.split(":")[1]) if type(x) == str else x)

In [ ]:
party4["time"] = party4["party_end_week"]*7*24*60 \
                 +party4["party_end_day"]*24*60 \
                 +party4["party_end_time"].map(lambda x : int(x.split(":")[0]) if type(x) == str else x)*60 \
                 +party4["party_end_time"].map(lambda x : int(x.split(":")[1]) if type(x) == str else x) \
                 -party4["week"]*7*24*60 \
                 -party4["day"]*24*60 \
                 -party4["party_start_time"].map(lambda x : int(x.split(":")[0]) if type(x) == str else x)*60 \
                 -party4["party_start_time"].map(lambda x : int(x.split(":")[1]) if type(x) == str else x) 

In [ ]:
party4 = party4.drop(["party_start_time", "party_end_week", "party_end_day", "party_end_time"], axis=1)
party4 = party4.sort_values(by = ["acc_id", "week", "day", "hour", "minute"])[["acc_id", "week", "day", "hour", "minute", "time"]]
party4.head()

In [ ]:
trade6.head(20)

In [ ]:
party4.head(20)

In [ ]:
result = pd.concat([trade6, party4])
result.head(20)

In [ ]:
result = result[["acc_id","week","day","hour", "minute", "time", "item_amount", "item_type", "source"]].sort_values(by= ["acc_id","week","day","hour", "minute"])
result.head(50)

In [ ]:
#activity 횟수의 분포
result.groupby("acc_id").count().groupby("week").count().head()

# user vector 만들기

In [ ]:
activity2 = activity.rename(columns = {"wk":"week"})
activity2.head()

## payment

### 각 주의 충전금액

In [ ]:
payment.sort_values("acc_id").head(20)

In [ ]:
activity_payment = pd.merge(activity2, payment, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment.head()

## party

### 각 주의 party 횟수

In [ ]:
party5 = party4.groupby(["acc_id", "week"]).count()[["day"]].rename(columns ={"day" : "party_count"}).reset_index()
party5.head(20)

In [ ]:
activity_payment_party = pd.merge(activity_payment, party5, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party.head(20)

In [ ]:
print(activity_payment_party.isnull().count()[-1])
activity_payment_party.isnull().any()

In [ ]:
activity_payment_party = activity_payment_party.fillna(0)

### 각 주의 party시간 총합

In [ ]:
party6 = party4.groupby(["acc_id", "week"]).sum()[["time"]].reset_index().rename(columns = {"time": "party_time_total"})
party6.head(20)

In [ ]:
activity_payment_party2 = pd.merge(activity_payment_party, party6, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party2.head(20)

In [ ]:
activity_payment_party2 = activity_payment_party2.fillna(0)

### 각 주의 party시간 평균

In [ ]:
party7 = party4.groupby(["acc_id", "week"]).mean()[["time"]].reset_index().rename(columns = {"time": "party_time_mean"})
party7.head(20)

In [ ]:
activity_payment_party3 = pd.merge(activity_payment_party2, party7, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party3.head(20)

In [ ]:
activity_payment_party3 = activity_payment_party3.fillna(0)

## trade

### 각 주의 trade횟수

In [ ]:
trade7 = trade6.groupby(["acc_id", "week"]).count()[["day"]].reset_index().rename(columns = {"day": "trade_count"})
trade7.head()

In [ ]:
activity_payment_party_trade = pd.merge(activity_payment_party3, trade7, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party_trade.head()

In [ ]:
activity_payment_party_trade = activity_payment_party_trade.fillna(0)

### 각 주의 trade 물품 횟수

In [ ]:
trade8 = trade6[["acc_id", "week", "item_type", "item_amount"]]
trade8 = trade8.groupby(["acc_id","week","item_type"]).count().reset_index()
trade8.head(15)

In [ ]:
trade8 = trade8.groupby(['acc_id','week', 'item_type'])['item_amount'].aggregate('first').unstack().reset_index().fillna(0)
trade8 .head(15)

In [ ]:
activity_payment_party_trade2 = pd.merge(activity_payment_party_trade, trade8, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party_trade2.head()

In [ ]:
col_dic = {"accessory" : "trade_accessory_cnt" ,
           "costume": "trade_costume_cnt",
           "gem": "trade_gem_cnt",
           "grocery": "trade_grocery_cnt",
           "money": "trade_money_cnt",
           "weapon": "trade_weapon_cnt"}

activity_payment_party_trade2 = activity_payment_party_trade2.rename(columns = col_dic)
activity_payment_party_trade2 = activity_payment_party_trade2.fillna(0)

### 각 주의 trade 물품 총합(정규화 문제)

In [ ]:
trade9 = trade6[["acc_id", "week", "item_type", "item_amount"]]
trade9 = trade9.groupby(["acc_id","week","item_type"]).sum().reset_index()
trade9.head(10)

In [ ]:
trade9 = trade9.groupby(['acc_id','week', 'item_type'])['item_amount'].aggregate('first').unstack().reset_index().fillna(0)
trade9.head(15)

In [ ]:
activity_payment_party_trade3 = pd.merge(activity_payment_party_trade2, trade9, how='left', on=['acc_id', 'week']).sort_values(by = ["acc_id", "week"])
activity_payment_party_trade3.head()

In [ ]:
col_dic = {"accessory" : "trade_accessory_total" ,
           "costume": "trade_costume_total",
           "gem": "trade_gem_total",
           "grocery": "trade_grocery_total",
           "money": "trade_money_total",
           "weapon": "trade_weapon_total"}

activity_payment_party_trade3 = activity_payment_party_trade3.rename(columns = col_dic)
activity_payment_party_trade3 = activity_payment_party_trade3.fillna(0)

In [ ]:
activity_payment_party_trade3.head(10)

### 각 주의 trade 물품 평균(아직 추가해주지 않음- 정규화 문제)

In [ ]:
trade10 = trade6[["acc_id", "week", "item_type", "item_amount"]]
trade10 = trade10.groupby(["acc_id","week","item_type"]).mean()
trade10.head(10)

# 정규화

In [ ]:
activity_payment_party_trade3.columns

In [ ]:
user_vector = activity_payment_party_trade3[['acc_id', 'week','cnt_dt', 'play_time', 'npc_exp', 'npc_hongmun',
       'quest_exp', 'quest_hongmun', 'item_hongmun', 'game_combat_time',
       'get_money', 'duel_cnt', 'duel_win', 'partybattle_cnt',
       'partybattle_win', 'cnt_enter_inzone_solo', 'cnt_enter_inzone_light',
       'cnt_enter_inzone_skilled', 'cnt_enter_inzone_normal', 'cnt_enter_raid',
       'cnt_enter_raid_light', 'cnt_enter_bam', 'cnt_clear_inzone_solo',
       'cnt_clear_inzone_light', 'cnt_clear_inzone_skilled',
       'cnt_clear_inzone_normal', 'cnt_clear_raid', 'cnt_clear_raid_light',
       'cnt_clear_bam', 'normal_chat', 'whisper_chat', 'district_chat',
       'party_chat', 'guild_chat', 'faction_chat', 'cnt_use_buffitem',
       'gathering_cnt', 'making_cnt', 'payment_amount', 'party_count',
       'party_time_total', 'party_time_mean', 'trade_count',
       'trade_accessory_cnt', 'trade_costume_cnt', 'trade_gem_cnt',
       'trade_grocery_cnt', 'trade_money_cnt', 'trade_weapon_cnt',
       'trade_accessory_total', 'trade_costume_total', 'trade_gem_total',
       'trade_grocery_total', 'trade_money_total', 'trade_weapon_total']].sort_values(['acc_id', 'week'])
user_vector.head(20)

In [ ]:
user_vector.isnull().any()

In [ ]:
user_vector["cnt_dt"] = (user_vector["cnt_dt"] - user_vector["cnt_dt"].mean()) / user_vector["cnt_dt"].std()

user_vector["party_count"] = (user_vector["party_count"] - user_vector["party_count"].mean()) / user_vector["party_count"].std()
user_vector["party_time_total"] = (user_vector["party_time_total"] - user_vector["party_time_total"].mean()) / user_vector["party_time_total"].std()
user_vector["party_time_mean"] = (user_vector["party_time_mean"] - user_vector["party_time_mean"].mean()) / user_vector["party_time_mean"].std()
user_vector["party_time_total"] = (user_vector["party_time_total"] - user_vector["party_time_total"].mean()) / user_vector["party_time_total"].std()

user_vector["trade_count"] = (user_vector["trade_count"] - user_vector["trade_count"].mean()) / user_vector["trade_count"].std()
user_vector["trade_accessory_cnt"] = (user_vector["trade_accessory_cnt"] - user_vector["trade_accessory_cnt"].mean()) / user_vector["trade_accessory_cnt"].std()
user_vector["trade_costume_cnt"] = (user_vector["trade_costume_cnt"] - user_vector["trade_costume_cnt"].mean()) / user_vector["trade_costume_cnt"].std()
user_vector["trade_gem_cnt"] = (user_vector["trade_gem_cnt"] - user_vector["trade_gem_cnt"].mean()) / user_vector["trade_gem_cnt"].std()
user_vector["trade_grocery_cnt"] = (user_vector["trade_grocery_cnt"] - user_vector["trade_grocery_cnt"].mean()) / user_vector["trade_grocery_cnt"].std()
user_vector["trade_money_cnt"] = (user_vector["trade_money_cnt"] - user_vector["trade_money_cnt"].mean()) / user_vector["trade_money_cnt"].std()
user_vector["trade_weapon_cnt"] = (user_vector["trade_weapon_cnt"] - user_vector["trade_weapon_cnt"].mean()) / user_vector["trade_weapon_cnt"].std()

In [ ]:
user_vector.head()

In [ ]:
user_vector.to_csv("user_vector_test.csv")